we are using data from https://github.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings

we are also augmenting the training dataset using nlpaug from https://github.com/makcedward/nlpaug

In [31]:
import pandas as pd
# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import torch

In [37]:
raw_df = pd.read_csv("./data/depression_dataset_reddit_cleaned.csv")


In [47]:
train = pd.read_csv('data/train.tsv', delimiter='\t')
test = pd.read_csv('data/test.tsv', delimiter='\t')
dev = pd.read_csv('data/dev.tsv', delimiter='\t')

In [48]:
test

,Pid,text,label
0,test_pid_1,Im scared : This is it. I lie to myself every ...,moderate
1,test_pid_2,New to this but just wanted to vent : I just f...,moderate
2,test_pid_3,I’m sad : It’s kinda always been an issue. I w...,moderate
3,test_pid_4,Lonely but not alone. : All of my immediately ...,moderate
4,test_pid_5,This year has been trash. : I dont know why I’...,moderate
...,...,...,...
3240,test_pid_3241,"Feeling lonely. : Hi reddit, I haven’t posted ...",severe
3241,test_pid_3242,When would suicide be right? : So I got back f...,severe
3242,test_pid_3243,Lowest I’ve ever been ever. : To make a long s...,severe
3243,test_pid_3244,Does the Toxoplasma Gondii ruined my life ? (f...,severe


In [49]:
dev

,PID,text,label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate
2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate
4,dev_pid_5,The world only cares about beautiful people : ...,moderate
...,...,...,...
4491,dev_pid_4492,Aren’t we all just tired? : I’ve been depresse...,severe
4492,dev_pid_4493,NEED HELP COPING : I had my life pretty much f...,severe
4493,dev_pid_4494,Qutting Zoloft Cold Turkey : I was on 75 mg se...,severe
4494,dev_pid_4495,Crying : I’m coming off my antidepressants and...,severe


In [50]:
print(test[test['label'] == 'severe'])

                Pid                                               text   label
3017  test_pid_3018  Why I haven't given up : Last night I was feel...  severe
3018  test_pid_3019  Don't know how to get better when I can't talk...  severe
3019  test_pid_3020  I want to give up : Two medications in and sti...  severe
3020  test_pid_3021  Pristiq and Wellbutrin : Hello! I'm curious ha...  severe
3021  test_pid_3022  Other people can’t solve YOUR problems! : I re...  severe
...             ...                                                ...     ...
3240  test_pid_3241  Feeling lonely. : Hi reddit, I haven’t posted ...  severe
3241  test_pid_3242  When would suicide be right? : So I got back f...  severe
3242  test_pid_3243  Lowest I’ve ever been ever. : To make a long s...  severe
3243  test_pid_3244  Does the Toxoplasma Gondii ruined my life ? (f...  severe
3244  test_pid_3245  Getting the correct diagnosis : Currently goin...  severe

[228 rows x 3 columns]


In [51]:
print(dev[dev['label'] == 'severe'])

               PID                                               text   label
4136  dev_pid_4137  I Just Realized Why I do Poorly in School Mome...  severe
4137  dev_pid_4138  my libido is extremely low and id like to fix ...  severe
4138  dev_pid_4139  How can I [F30] stop taking paroxetine gradual...  severe
4139  dev_pid_4140  I just took 5 more prozac pills than i should’...  severe
4140  dev_pid_4141  Dreams/nightmares. : Hello I am taking propano...  severe
...            ...                                                ...     ...
4491  dev_pid_4492  Aren’t we all just tired? : I’ve been depresse...  severe
4492  dev_pid_4493  NEED HELP COPING : I had my life pretty much f...  severe
4493  dev_pid_4494  Qutting Zoloft Cold Turkey : I was on 75 mg se...  severe
4494  dev_pid_4495  Crying : I’m coming off my antidepressants and...  severe
4495  dev_pid_4496  Seeking for advice on how to overcome and deal...  severe

[360 rows x 3 columns]


In [52]:
set(train['label'])

{'moderate', 'not depression', 'severe'}

In [53]:
def check_dataset(df):
    total = len(df)
    print('length: ', total)
    num_non_dep = len(df[df['label'] == 'not depression'])
    num_mod = len(df[df['label'] == 'moderate'])
    num_seve = len(df[df['label'] == 'severe'])
    print('% non depression: ', f"{num_non_dep/total:.4f}")
    print('% moderate: ', f"{num_mod/total:.4f}")
    print('% severe: ', f"{num_seve/total:.4f}")

In [54]:
check_dataset(train)

length:  8891
% non depression:  0.2217
% moderate:  0.6770
% severe:  0.1013


In [55]:
check_dataset(dev)

length:  4496
% non depression:  0.4070
% moderate:  0.5129
% severe:  0.0801


In [56]:
check_dataset(test)

length:  3245
% non depression:  0.2613
% moderate:  0.6684
% severe:  0.0703


## POSTSCRIPT: DATA LEAKAGE

In [6]:
# combining and resplitting the dataset
print(train.columns)
print(test.columns)
print(dev.columns)

train = train.rename(columns={"Text_data": "text", "Label": "label"})
test = test.rename(columns={"text data": "text", "Class labels": "label", "Pid": "PID"})
dev = dev.rename(columns={"Text data": "text", "Label": "label"})

print(train.columns)
print(test.columns)
print(dev.columns)

combined = pd.concat([train, test, dev], axis=0)

Index(['PID', 'text', 'label'], dtype='object')
Index(['PID', 'text', 'label'], dtype='object')
Index(['PID', 'text', 'label'], dtype='object')
Index(['PID', 'text', 'label'], dtype='object')
Index(['PID', 'text', 'label'], dtype='object')
Index(['PID', 'text', 'label'], dtype='object')


In [6]:
from sklearn.model_selection import train_test_split

# First split off test set
train_val, test = train_test_split(combined, test_size=0.15, random_state=42, stratify=combined['label'])

# Then split train/val
train, val = train_test_split(train_val, test_size=0.1765, random_state=42, stratify=train_val['label'])
# 0.1765 ensures val is 15% of the total

print(len(train), len(val), len(test))


11641 2496 2495


In [7]:
print(len(val[val["label"]=='moderate']))
print(len(test[test["label"]=='moderate']))

1573
1572


In [8]:
train.to_csv('data/reshuffle_train.tsv', sep='\t')
val.to_csv('data/reshuffle_dev.tsv', sep='\t')
test.to_csv('data/reshuffle_test.tsv', sep='\t')

In [11]:
# balance out the train datset
moderate_df = train[train['Label'] == 'moderate']
keep_moderate = moderate_df.sample(frac=0.3, random_state=42)
non_moderate_df = train[train['Label'] != 'moderate']
balanced_df = pd.concat([non_moderate_df, keep_moderate]).reset_index(drop=True)
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
balanced_df.to_csv('data/balanced_train.tsv', sep='\t')

In [4]:
print(len(train))

print(len(train[train['Label'] == 'moderate']) / len(train))
print(len(train[train['Label'] == 'not depression']) / len(train))
print(len(train[train['Label'] == 'severe']) / len(train))

8891
0.675289618715555
0.2232594758744798
0.10145090540996514


In [5]:
print(len(test))
print(len(test[test['Class labels'] == 'moderate']) / len(test))
print(len(test[test['Class labels'] == 'not depression']) / len(test))
print(len(test[test['Class labels'] == 'severe']) / len(test))

3245
0.6684129429892142
0.2613251155624037
0.07026194144838213


In [ ]:
# randomly remove 70% of moderate labels to balance out the classes

In [6]:
print(len(dev))
print(len(dev[dev['Label'] == 'moderate']) / len(dev))
print(len(dev[dev['Label'] == 'not depression']) / len(dev))
print(len(dev[dev['Label'] == 'severe']) / len(dev))

4496
0.5129003558718861
0.4070284697508897
0.0800711743772242


In [10]:
pd.set_option('display.max_colwidth', None)
# {'moderate', 'not depression', 'severe'}
train[train['Label'] == 'severe'].iloc[:10]

,PID,Text_data,Label
3372,train_pid_3373,"Falling in love with depression and OCD. Another nightmare times. : Hi, I've got depression and OCD, I'm being treated with sertraline (200mg) for about 0.5 year. I just met someone that I actually fallen in love with. She has anxiety disorder, I think she's been through a hard time becuase of her scars and amount of ""close calls"" overdosing the pills. After me being this stupid guy before, I once tried to help someone with the same thing. As you can guess this 1st person is still alive becuase of me but she wasn't herself anymore. Now when the next opportunity comes, I just feel like It already happend and I don't want to engage becuase it will end the same way. I just feel worse and worse without talking to her but I also feel like nobody, like someone you couldn't like or even shouldn't like. I don't know how to live with that feeling, I know that I'm a fucking good person to the top, but I also know not everyone is like me and sometimes they will not know the effort put to just being close to someone, to save her life. I'm scared, this time not from OCD and depression, this time becuase of my own memories coming back, this time becuase of love I just can't stand as such emotional person. What should I really do? Help me seek some reasonable answers. I'm not typing this to make myself the worst person on the earth, I just write to get some help organizing my thoughts and my brain. Please help.",severe
3389,train_pid_3390,"I’m so empty : I wake up to the malevolent glare of life once again. I ponder if I want to get out of my bed, let alone my room. I look around and see nothing but bad memories, and empty promises. I begin to imagine myself with a needle in my hand. To once again feel the warm embrace of heroin, and if lucky. Maybe even death. My reminiscing is cut short by a 5 year old girl bursting into my room with a smile on her face. Felicity. My baby sister is one of the few reasons I get up in the morning. And to see the familiar, and chubby face brought me nothing but what I guess was joy. It is now time for me to put on the mask. \n\n\nI’ve learned many things about depression in the past 3 years. One of those things is to never grant anybody access to my emotions. The trauma, and pain I have experienced from the hospitals I’ve been subjected to, are second to none. Absolute agony. So I wear a metaphorical mask, one of energy, and joy. I walk down the carpet stairs whilst holding Felicity’s hand, and I am instantly greeted with the expectations for the day. Today I must play with Felicity, clean my room, do my laundry, clean my bathroom, clean the kitchen, and keep myself from killing myself, or getting high. I also cannot talk to either of my parents about how I’m doing, because I’d be bothering them. \n\nI don’t do most of these things. I simply take my medicine, and watch my humanity, and emotions slip away with a gulp. I do this everyday. I fall asleep as the medicine kicks in, and takes my energy. I do this everyday. I eat. And eat, and eat, and eat, until I feel like throwing up. I do nothing with my life. The only thing I do is listen to music. I love music. I truly adore all kinds of it. I love learning about music. I love listening to music. I play 4 instruments, and produce music. But I don’t anymore \n\nDepression has taken away my future. I don’t have the drive to create music. So I just sit. I sit. I eat, I sleep, and I do it all again. I’m so alone, I’m so sad. I’m so empty.\n\n\nNot quite sure what this post is, but I feel like shit, and don’t want to live like this anymore. Any advice is appreciated",severe
3392,train_pid_3393,"I can't kill myself because of my parents : I just hate it. My parents are often annoying and rude with me, but when they see news on the tv where young people died they often say that they wouldn't bear if I died and that they would kill themself.\n\nAnd because of this I can't kill myself. I would feel too guilty. I've now been in

Augment only the training set to increase the representation of not depressed datapoints.
Use Roberta Bert base to augment the not depressed

In [7]:
not_depressed_train = train[train['Label'] == 'not depression']

In [8]:
text_to_augment = not_depressed_train['Text_data'].tolist()

labels_of_augment = not_depressed_train['Label'].tolist()
labels_of_augment.extend(labels_of_augment)

In [9]:
torch.cuda.is_available()

True

Code for augmenting the data

In [10]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute", device='cuda')
augmented_text = aug.augment(text_to_augment)
print("Original:")
print(text_to_augment)
print("Augmented Text:")
print(augmented_text)

Original:
["Hows everyone doing on this new years eve? : I know as well as everyone new years makes you think back and regret and wonder why you're still here one more year..\n\nWell i'm spending new years alone at home and I'm just wondering how everyone is doing and if you need a place to vent or talk this can be a safe place for you. \n\nHappy new year everyone. We made it and maybe things will get better.", 'Goodbye : [removed]', 'thank you : i’m not the best at this so this may come off as weird, but i barely know any of you but we feel as if we really look out for each other. it’s nice to know that others are going through the same issues and we can help each other to continue our fight. here’s to the new year everyone.', "Have a happy near year.... : I'm spending this new years alone and in bed. \n\nI hope you are not doing the same. I hope you can have fun today if your reading this, next year is gonna be lit, dont give up on yourself. Your all you got in this life. Love you gu

In [17]:
original_train_data = pd.read_csv('data/train.tsv', delimiter='\t')
original_train_data.columns
train_text = original_train_data['Text_data']
train_labels = original_train_data['Label']

In [ ]:
train_text.extend(augmented_text)
train_labels.extend(labels_of_augment)

In [18]:
df = pd.DataFrame({'text': train_text, 'label': train_labels})
df.to_csv('data/train_aug.tsv', sep='\t')

In [19]:
df

,text,label
0,Waiting for my mind to have a breakdown once t...,moderate
1,My new years resolution : I'm gonna get my ass...,moderate
2,New year : Somone else Feeling like 2020 will ...,moderate
3,"My story I guess : Hi, Im from Germany and my ...",moderate
4,Sat in the dark and cried myself going into th...,moderate
...,...,...
8886,Ways to reverse memory loss from depression? :...,severe
8887,A Comprehensive Guide To Slowly Getting Better...,severe
8888,I don’t think college is right for me : TW: su...,severe
8889,Please help: Severe insomnia affecting me in m...,severe
